In [1]:
import numpy as np
import torch
import torchvision as tvis
import torch.nn as nn
from torchsummary import summary
import matplotlib.pyplot as plt
from torch.utils.data.sampler import SubsetRandomSampler
import torch.nn.functional as F
import torchtext

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')
torch.cuda.is_available()

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


False

In [12]:
data_dir = "./gdrive/My Drive/coding_projects/transformer/wmt14"
#torchtext.datasets.WMT14.download(data_dir)
#!python3 -m spacy download de
#!python3 -m spacy download en

In [13]:
def get_loader(data_dir,
               batch_size,
               random_seed,
               valid_size=0.2,
               shuffle=True,
               num_workers=2):


    from torchtext import data
    from torchtext import datasets
    from torchtext.datasets import TranslationDataset
    import re
    import spacy

    spacy_de = spacy.load('de')
    spacy_en = spacy.load('en')

    def tokenize_de(text):
        return [tok.text for tok in spacy_de.tokenizer(text)]

    def tokenize_en(text):
        return [tok.text for tok in spacy_en.tokenizer(text)]


    BOS_WORD = '<s>'
    EOS_WORD = '</s>'
    BLANK_WORD = "<blank>"
    SRC = data.Field(tokenize=tokenize_de, pad_token=BLANK_WORD)
    TGT = data.Field(tokenize=tokenize_en, init_token=BOS_WORD, eos_token=EOS_WORD, pad_token=BLANK_WORD)

    MAX_LEN = 100
    print('splits')
    train, val, test = datasets.WMT14.splits(
                                  exts=('.de', '.en'),
                                  train='train.tok.clean.bpe.32000', 
                                  validation='newstest2013.tok.bpe.32000', 
                                  test='newstest2014.tok.bpe.32000',
                                  fields=(SRC, TGT),
                                  filter_pred=lambda x: len(vars(x)['src']) <= MAX_LEN and len(vars(x)['trg']) <= MAX_LEN,
                                  root=data_dir)

    print(train.fields)
    print(len(train))
    print(vars(train[0]))
    print(vars(train[100]))


In [ ]:
get_loader(data_dir,0,0)

splits
downloading wmt16_en_de.tar.gz
downloading from Google Drive; may take a few minutes


wmt16_en_de.tar.gz: 503MB [00:06, 79.0MB/s]


In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F

"""Attention layer.
"""
class Attention(nn.Module):
    def __init__(self, h=4, d_model=128, d_k=32, d_v=32):
        super(Attention, self).__init__()
        self.h = h
        self.d_model = d_model
        self.d_k = d_k
        self.d_v = d_v
        
        self.W_q = torch.rand(size=(h, d_model, d_k),requires_grad=True)
        self.W_k = torch.rand(size=(h, d_model, d_k),requires_grad=True)
        self.W_v = torch.rand(size=(h, d_model, d_v),requires_grad=True)
        self.W_o = torch.rand(size=(int(h*d_v),d_model),requires_grad=True)

    def forward(self, x):
        # x in [BS,seq_len,d_model]
        Q = torch.einsum('ijk,lkm->ijlm', x, self.W_q) #[BS,seq_len,h,d_k]
        K = torch.einsum('ijk,lkm->ijlm', x, self.W_k) #[BS,seq_len,h,d_k]
        V = torch.einsum('ijk,lkm->ijlm', x, self.W_v) #[BS,seq_len,h,d_v]
        QK_T = torch.einsum('ijkl,ijkl->ijk',Q,K)                    #[BS,seq_len,h]
        att  = torch.einsum('ijk,ijkl->ijkl', F.softmax(QK_T),V) #[BS,seq_len,h,d_v]
        att_conc = att.view(att.shape[0],att.shape[1],-1)            #[BS,seq_len,h*d_v]
        mh_att = torch.einsum('ijk,kl->ijl',att_conc,self.W_o)       #[BS,seq_len,d_model]

        return mh_att

x = torch.rand(size=(24,10,128))
mod = Attention()
out = mod(x)

In [10]:
att = Attention()


In [11]:
x = torch.rand(size=(12,10,128))

In [12]:
p = att.forward(x)

RuntimeError: ignored